In [1]:
import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib import pyplot as plt
from thefuzz import fuzz
import easyocr
from pylab import rcParams
from IPython.display import Image
import os
from PIL import Image
import re
import requests
from math import radians, cos, sin, asin, sqrt
import csv
from pytesseract import pytesseract
import tempfile
import shutil

C:\Users\madha\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def crop_footer(image_path, footer_height):
    image = Image.open(image_path)
    width, height = image.size
    left = 0
    upper = height - footer_height
    right = width
    lower = height
    cropped_image = image.crop((left, upper, right, lower))
    cropped_image.save("cropped_image.png")
    
    
def crop_header(image_path, header_height):
    image = Image.open(image_path)
    width, height = image.size
    left = 0
    upper = 0
    right = width
    lower = header_height
    cropped_image = image.crop((left, upper, right, lower))
    cropped_image.save("cropped_image1.png")

In [3]:
def text_extract(img1):
    path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

    # Opening the image & storing it in an image object
    img = Image.open(img1)
    pytesseract.tesseract_cmd = path_to_tesseract

    text = pytesseract.image_to_string(img)
    return text


# Function to retrieve pincode from extracted text
def extract_pincode(text):
    pincode_regex = r"\b\d{6}\b"
    pincode_match = re.search(pincode_regex, text)
    if pincode_match:
        pincode = pincode_match.group()
        return pincode
    else:
        return None

In [4]:
def geocode_postal_code(postal_code):
    url = f"https://nominatim.openstreetmap.org/search"
    params = {
        "q": postal_code,
        "format": "json",
        "addressdetails": 1,
        "limit": 10
    }
    response = requests.get(url, params=params,verify=False)
    if response.ok:
        data = response.json()
        if data:
           # print(data)
            for result in data:
                if 'address' in result and 'country_code' in result['address'] and result['address']['country_code'] == 'in':
                    place_name = result['display_name']
                    latitude = float(result['lat'])
                    longitude = float(result['lon'])
                    return place_name, latitude, longitude
    return None, None, None

In [6]:
data = pd.read_csv('patient_addresses_new.csv')
data.head()

,Hospital_Pincode,Patient_Pincode
0,122001,110025
1,110026,508001
2,110035,560025
3,122025,122001
4,201301,201306


In [7]:
image_path = "report1.jpg"
footer_height = 200
header_height = 150

crop_footer(image_path, footer_height)
footer_img_path = 'cropped_image.png'
footer_text = text_extract(footer_img_path)

# Crop the header and extract text
crop_header(image_path, header_height)
header_img_path = 'cropped_image1.png'
header_text = text_extract(header_img_path)


combined_text = header_text + " " + footer_text
pincode_pharmacy = extract_pincode(combined_text)
data['pharmacy_pincode'] = pincode_pharmacy

In [8]:
from geopy.distance import geodesic

def calculate_distance_between_pincode(data, col1, col2):

    for index, row in data.iterrows():
        column1 = row[col1]
        column2 = row[col2]

        place_name1, latitude1, longitude1 = geocode_postal_code(column1)
        place_name2, latitude2, longitude2 = geocode_postal_code(column2)

        distance = geodesic((latitude1, longitude1), (latitude2, longitude2)).km

        new_column_name = f"{col1} to {col2}"
        data.at[index, new_column_name] = distance


In [9]:
col1="Hospital_Pincode"
col2="pharmacy_pincode"
calculate_distance_between_pincode(data,col1,col2)
data.head()

C:\Users\madha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nominatim.openstreetmap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\madha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nominatim.openstreetmap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\madha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nominatim.openstreetmap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\madha\anacond

,Hospital_Pincode,Patient_Pincode,pharmacy_pincode,Hospital_Pincode to pharmacy_pincode
0,122001,110025,508001,1284.670687
1,110026,508001,508001,1303.246676
2,110035,560025,508001,1305.485704
3,122025,122001,508001,8875.062752
4,201301,201306,508001,1293.546211


In [10]:
col1="Patient_Pincode"
col2="pharmacy_pincode"
calculate_distance_between_pincode(data,col1,col2)
data.head()

C:\Users\madha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nominatim.openstreetmap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\madha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nominatim.openstreetmap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\madha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nominatim.openstreetmap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\madha\anacond

,Hospital_Pincode,Patient_Pincode,pharmacy_pincode,Hospital_Pincode to pharmacy_pincode,Patient_Pincode to pharmacy_pincode
0,122001,110025,508001,1284.670687,1289.619120
1,110026,508001,508001,1303.246676,0.000000
2,110035,560025,508001,1305.485704,486.106852
3,122025,122001,508001,8875.062752,1284.670687
4,201301,201306,508001,1293.546211,1280.777124


In [11]:
col1="Hospital_Pincode"
col2="Patient_Pincode"
calculate_distance_between_pincode(data,col1,col2)
data.head()

C:\Users\madha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nominatim.openstreetmap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\madha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nominatim.openstreetmap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\madha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nominatim.openstreetmap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\madha\anacond

,Hospital_Pincode,Patient_Pincode,pharmacy_pincode,Hospital_Pincode to pharmacy_pincode,Patient_Pincode to pharmacy_pincode,Hospital_Pincode to Patient_Pincode
0,122001,110025,508001,1284.670687,1289.619120,26.178471
1,110026,508001,508001,1303.246676,0.000000,1303.246676
2,110035,560025,508001,1305.485704,486.106852,1740.470002
3,122025,122001,508001,8875.062752,1284.670687,8747.750794
4,201301,201306,508001,1293.546211,1280.777124,17.754436


In [12]:
data.to_csv('file1.csv')